In [14]:
import streamlit as st
from glob import glob
from bs4 import BeautifulSoup
from tqdm import tqdm
from html2text import html2text
from collections import defaultdict
from fuzzywuzzy import fuzz

C:\Users\dakil\miniconda3\envs\bg3_tools\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [23]:
dialogue_files_latter = list(glob("data/1.2 - Patch 2/**/*.html", recursive=True))
dialogue_files_former = list(glob("data/1.0 - Launch Day/**/*.html", recursive=True))

former_lines_dict = defaultdict(set)
latter_lines_dict = defaultdict(set)
for fn in tqdm(dialogue_files_former):
    file = open(fn, "r", encoding='latin-1')
    index = file.read()
    source = BeautifulSoup(index, 'html')
    div_elements = source.find_all('div', class_="npc")
    for div_element in div_elements:
        character = div_element.text
        dialog_span = div_element.find_next('span', class_='dialog')
        former_lines_dict[character].add(html2text(str(dialog_span)).strip())

for fn in tqdm(dialogue_files_latter):
    file = open(fn, "r", encoding='latin-1')
    index = file.read()
    source = BeautifulSoup(index, 'html')
    div_elements = source.find_all('div', class_="npc")
    for div_element in div_elements:
        character = div_element.text
        dialog_span = div_element.find_next('span', class_='dialog')
        latter_lines_dict[character].add(html2text(str(dialog_span)).strip())

0it [00:00, ?it/s]
100%|██████████| 9218/9218 [02:35<00:00, 59.17it/s] 


In [7]:
all_characters = set(list(former_lines_dict.keys()) + list(latter_lines_dict.keys()))

In [22]:
for character in all_characters:
    lines = [l for l in latter_lines_dict[character] if l not in former_lines_dict[character]]
    new_lines = set()
    changed_lines = []
    for line in lines:
        for old_line in former_lines_dict[character]:
            if fuzz.ratio(line, old_line) > 80:
                changed_lines.append((old_line, line))
                break
        else:
            new_lines.add(line)
    
    if any([len(new_lines) > 0, len(changed_lines) > 0]):
        print(character)
        print("New Lines", new_lines)
        print("changed_lines", changed_lines)

Mizora
New Lines set()
changed_lines [("Just because you're not my pet doesn't mean you're not useful.", "Just because he's not my pet doesn't mean he's not useful.")]
Orin
New Lines set()
changed_lines [("Bah! But he must lead the murdermarch to Baldur's Grave.", "But he must lead the murdermarch to Baldur's Grave.")]
Wulbren Bongle
New Lines set()
changed_lines [('Efficent? Good.', 'Efficient? Good.')]
Shadowheart
New Lines {"You know... I had my suspicions, but I didn't want to act on them. And now it\nseems I won't have to."}
changed_lines [("But imagine what we might achieve if we channeled some of that hostility back\nat our real foes instead of each other. They wouldn't stand a chance.", "But imagine what we might achieve if we channelled some of that hostility back\nat our real foes instead of each other. They wouldn't stand a chance."), ("I'll give you all that I can. Just don't call it. love.", "I'll give you all that I can. Just don't call it love.")]
Astarion
New Lines set(

In [28]:
patches = [v.replace("\\", '/').split('/')[1] for v in glob("data/*") if "Parser" not in v]

In [29]:
print(patches)

['1.0 - Launch Day', '1.2 - Patch 2']
